In [1]:
import os
import zarr
import napari
import numpy as np
import copick
import copick_utils
from magicgui import magicgui
from napari.types import ImageData, LabelsData
from pathlib import Path
from cellcanvas_spp.segmentation import superpixels

try:
    # DATA_DIR = Path("/Users/kharrington/git/cellcanvas/superpixels/notebooks/my_synthetic_data_10439_dataportal.json")
    DATA_DIR = Path(os.environ["COPICK_DATA"])
except KeyError:
    raise ValueError(
        "Please set the COPICK_DATA environment variable to point to the data directory\n\n"
        "$ export COPICK_DATA=</path/to/copick/data>"
    )

@magicgui(auto_call=True, sigma={"widget_type": "FloatSlider", "min": 0, "max": 10}, h_minima={"widget_type": "FloatSlider", "min": 0, "max": 50})
def _spp_widget(image: ImageData, sigma: float = 4, h_minima: float = 0.001) -> LabelsData:
    return superpixels(image, sigma=sigma, h_minima=h_minima)

_spp_widget.h_minima._widget._readout_widget.setDecimals(4)

def get_segmentation_array(copick_run, segmentation_name, voxel_spacing=10, is_multilabel=True):
    seg = copick_run.get_segmentations(is_multilabel=is_multilabel, name=segmentation_name, voxel_size=voxel_spacing)
    if len(seg) == 0:
        return None
    segmentation = zarr.open(seg[0].zarr().path, mode="r")['0']
    return segmentation[:]

def segment_superpixels(example_run, voxel_spacing=10, interactive: bool = False):
    """
    This function handles the segmentation logic. If the segmentation exists in copick, it will be skipped.
    """
    segmentation_name = "superpixelSegmentation"

    # Check if the segmentation already exists
    seg = get_segmentation_array(example_run, segmentation_name, voxel_spacing=voxel_spacing, is_multilabel=True)
    if seg is not None:
        print(f"Segmentation '{segmentation_name}' already exists. Skipping segmentation.")
        return seg

    # Proceed with superpixel segmentation if it does not exist
    tomo_type = "wbp"
    tomogram = example_run.voxel_spacings[0].tomograms[0]

    # Open zarr
    z = zarr.open(tomogram.zarr())
    img = z["0"]  # Get the highest resolution scale

    if interactive:
        img = img[50:100, 180:360, 210:430]  # Cropping for interactive mode

    print("Loading image into memory ...")
    img = np.asarray(img)  # Loading into memory

    print("Segmenting superpixels ...")
    segm = superpixels(img, sigma=4, h_minima=0.0025)
    print("Done ...")

    # Save segmentation into copick
    print("Saving segmentation to copick...")
    new_seg = example_run.new_segmentation(voxel_spacing, segmentation_name, session_id="0", is_multilabel=True, user_id="cellcanvasSPP")
    segmentation_group = zarr.open_group(new_seg.path, mode="a")
    segmentation_group["0"] = segm
    print("Segmentation saved.")

    return segm

def prepare_and_run_segmentation(interactive: bool = False):
    """
    Prepare and run the segmentation by fetching data from copick.
    """
    config_file = DATA_DIR
    root = copick.from_file(config_file)

    run_name = "16193"
    example_run = root.get_run(run_name)
    voxel_spacing = 10

    # Check for existing segmentation or run segmentation if not found
    seg = segment_superpixels(example_run, voxel_spacing, interactive)

    # Open viewer and add image
    viewer = napari.Viewer()
    img = np.asarray(zarr.open(example_run.voxel_spacings[0].tomograms[0].zarr())["0"])  # Load image
    viewer.add_image(img)
    viewer.add_labels(seg, name="Superpixels")

    # Add ground truth
    base_seg = np.zeros_like(zarr.open(example_run.segmentations[0].zarr())["0"])
    for idx, seg in enumerate(example_run.segmentations):
        z = zarr.open(seg.zarr())["0"][:]
        base_seg = base_seg + (idx + 1) * z
    viewer.add_labels(base_seg)

    # If interactive mode, show the widget
    if interactive:
        viewer.window.add_dock_widget(_spp_widget, area="right")

    # napari.run()
    return viewer

# Run segmentation directly in a Jupyter notebook
viewer = prepare_and_run_segmentation(interactive=False)


Segmentation 'superpixelSegmentation' already exists. Skipping segmentation.


/Users/kharrington/.album/micromamba/envs/nesoi/lib/python3.10/site-packages/morphometrics_engine/measure.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
